<a href="https://colab.research.google.com/github/ngzh-luke/muict-498prj-ictchat/blob/main/src/model/Muict_Chatbot_Trainner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Note that this Notebook is base on this resource \
https://colab.research.google.com/drive/12dVqXZMIVxGI0uutU6HG9RWbWPXL3vts?usp=sharing

## Setup


In [ ]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Pre-process Dataset



Note that the question set have been augment via this tool beforehand
https://colab.research.google.com/drive/1WCY4QW1jkyMwOfhFuddZf3YkwEaLs218?usp=sharing

In [ ]:
import pandas as pd
from datasets import Dataset
from google.colab import files
import io


# from google.colab import drive
# drive.mount('/content/gdrive')
# data_dir = '/content/gdrive/MyDrive/Datasets/course_info_modified.csv'
# data = pd.read_csv(data_dir, encoding='utf-8')


# Upload file
uploaded = files.upload()
filename = list(uploaded.keys())[0]
data = pd.read_csv(io.BytesIO(uploaded[filename]), encoding='utf-8')


Saving training_data.csv to training_data (1).csv


In [ ]:
import pandas as pd
from datasets import Dataset

# Add prefixes to question and answer columns
question_prefix = '[Mahidol Information and Communication Technology Program] '
answer_prefix =  ''

# Format database for training
formatted_dataset = []
for _, row in data.iterrows():
    # Check if either Question or Answer is empty, then skip this row
    if pd.isnull(row['Question']) or pd.isnull(row['Answer']):
        continue

    formatted_row = f"### Input: {question_prefix}{row['Question']} \n### Output: {answer_prefix}{row['Answer']}"
    formatted_dataset.append(formatted_row)

# Convert dataset to Hugging Face's Dataset format
dataset = Dataset.from_dict({"text": formatted_dataset})

print(dataset)


Dataset({
    features: ['text'],
    num_rows: 780
})


,text
0,### Input: [Mahidol Information and Communicat...
1,### Input: [Mahidol Information and Communicat...
2,### Input: [Mahidol Information and Communicat...
3,### Input: [Mahidol Information and Communicat...
4,### Input: [Mahidol Information and Communicat...
...,...
775,### Input: [Mahidol Information and Communicat...
776,### Input: [Mahidol Information and Communicat...
777,### Input: [Mahidol Information and Communicat...
778,### Input: [Mahidol Information and Communicat...


No charts were generated by quickchart


## Loading Mistralai-7B or Llama2

In [ ]:
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
import torch
from huggingface_hub import notebook_login

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

# To switch to Llama2 replace this with "Llama2"
model_use = 'Llama'

if (model_use == 'Mistral'):
  model_name = "mistralai/Mistral-7B-Instruct-v0.2"
  output_model_name = "mistral-qlora-finetunined-MUICT-Chatbot"
else:
  model_name = "meta-llama/Llama-2-7b-chat-hf"
  output_model_name = "llama2-qlora-finetunined-MUICT-Chatbot"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Load tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
from peft import LoraConfig, get_peft_model

lora_alpha = 24
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

## Loading the trainer



We will use SFTTrainer to fine-tune the model because it **requires fewer resources and less memory**. SFTTrainer leverages Parameter-Efficient Fine-Tuning (PEFT), which allows you to train only the newly introduced parameters while keeping the pre-trained parameters frozen.










In [ ]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 1
gradient_accumulation_steps = 1
optim = "paged_adamw_32bit"
save_steps = 100
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 600
warmup_ratio = 0.03
lr_scheduler_type = "constant"
metric_for_best_model = "accuracy"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    metric_for_best_model=metric_for_best_model
)

In [ ]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

Map:   0%|          | 0/780 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

## Train the model

Now let's train the model! Simply call `trainer.train()`

In [ ]:
trainer.train()
# f14696f23e65b7bd541bb95882ab1bce609e2926

Step,Training Loss
10,2.630500
20,1.828700
30,1.484500
40,1.440000
50,1.086000
60,1.997000
70,1.221700
80,1.168100
90,1.162500
100,0.937700


TrainOutput(global_step=600, training_loss=1.0248469805717468, metrics={'train_runtime': 183.7986, 'train_samples_per_second': 3.264, 'train_steps_per_second': 3.264, 'total_flos': 1555882721869824.0, 'train_loss': 1.0248469805717468, 'epoch': 0.7692307692307693})

#Save model to Hugging Face

In [ ]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model
model_to_save.save_pretrained("outputs")

In [ ]:
lora_config = LoraConfig.from_pretrained('outputs')
model = get_peft_model(model, lora_config)

In [ ]:
model.push_to_hub(output_model_name)


README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/134M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/maunoi/llama2-qlora-finetunined-MUICT-Chatbot/commit/fca9a063eafc6c5c8a55bf39e3cba8d71f6f8a7f', commit_message='Upload model', commit_description='', oid='fca9a063eafc6c5c8a55bf39e3cba8d71f6f8a7f', pr_url=None, pr_revision=None, pr_num=None)